# Adding data to a data frame {#exr-add-to-dataframe-frog}

<hr>

In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade polars bebi103 watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    data_path = "https://s3.amazonaws.com/bebi103.caltech.edu/data/"
else:
    data_path = "../data/"
# ------------------------------

import polars as pl

<hr>

We continue working with the frog tongue data. Recall that the header comments in the data file contained information about the frogs.

In [2]:
!head -20 ../data/frog_tongue_adhesion.csv

# These data are from the paper,
#   Kleinteich and Gorb, Sci. Rep., 4, 5225, 2014.
# It was featured in the New York Times.
#    http://www.nytimes.com/2014/08/25/science/a-frog-thats-a-living-breathing-pac-man.html
#
# The authors included the data in their supplemental information.
#
# Importantly, the ID refers to the identifites of the frogs they tested.
#   I:   adult, 63 mm snout-vent-length (SVL) and 63.1 g body weight,
#        Ceratophrys cranwelli crossed with Ceratophrys cornuta
#   II:  adult, 70 mm SVL and 72.7 g body weight,
#        Ceratophrys cranwelli crossed with Ceratophrys cornuta
#   III: juvenile, 28 mm SVL and 12.7 g body weight, Ceratophrys cranwelli
#   IV:  juvenile, 31 mm SVL and 12.7 g body weight, Ceratophrys cranwelli
date,ID,trial number,impact force (mN),impact time (ms),impact force / body weight,adhesive force (mN),time frog pulls on target (ms),adhesive force / body weight,adhesive impulse (N-s),total contact area (mm2),contact area without mucus (m

So, each frog has associated with it an age (adult or juvenile), snout-vent-length (SVL), body weight, and species (either cross or *cranwelli*). For a tidy data frame, we should have a column for each of these values. Your task is to load in the data, and then add these columns to the data frame. For convenience, here is a data frame with data about each frog.

In [3]:
df_frog = pl.DataFrame(
    data={
        "ID": ["I", "II", "III", "IV"],
        "age": ["adult", "adult", "juvenile", "juvenile"],
        "SVL (mm)": [63, 70, 28, 31],
        "weight (g)": [63.1, 72.7, 12.7, 12.7],
        "species": ["cross", "cross", "cranwelli", "cranwelli"],
    }
)

Note: There are lots of ways to solve this problem. This is a good exercise in searching through the Polars documentation and other online resources, such as [Stack Overflow](https://stackoverflow.com/questions). Remember, much of your programming efforts are spent searching through documentation and the internet.

Finally, as a fun challenge, see if you can highlight the strike with the highest impact force for each frog in the data frame.

<br />

## Solution

<hr>

In [4]:
import great_tables

The most direct way is to use [built-in pd.merge() function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html). This function finds a common column between two `DataFrames`, and then uses that column to merge them, filling in values that match in the common column. This is exactly what we want.

In [5]:
# Load the data
df = pl.read_csv(os.path.join(data_path, 'frog_tongue_adhesion.csv'), comment_prefix='#')

# Perform merge
df = df.join(other=df_frog, on='ID')

Let's look at the `DataFrame` to make sure it has what we expect.

In [6]:
df.head()

date,ID,trial number,impact force (mN),impact time (ms),impact force / body weight,adhesive force (mN),time frog pulls on target (ms),adhesive force / body weight,adhesive impulse (N-s),total contact area (mm2),contact area without mucus (mm2),contact area with mucus / contact area without mucus,contact pressure (Pa),adhesive strength (Pa),age,SVL (mm),weight (g),species
str,str,i64,i64,i64,f64,i64,i64,f64,f64,i64,i64,f64,i64,i64,str,i64,f64,str
"""2013_02_26""","""I""",3,1205,46,1.95,-785,884,1.27,-0.29,387,70,0.82,3117,-2030,"""adult""",63,63.1,"""cross"""
"""2013_02_26""","""I""",4,2527,44,4.08,-983,248,1.59,-0.181,101,94,0.07,24923,-9695,"""adult""",63,63.1,"""cross"""
"""2013_03_01""","""I""",1,1745,34,2.82,-850,211,1.37,-0.157,83,79,0.05,21020,-10239,"""adult""",63,63.1,"""cross"""
"""2013_03_01""","""I""",2,1556,41,2.51,-455,1025,0.74,-0.17,330,158,0.52,4718,-1381,"""adult""",63,63.1,"""cross"""
"""2013_03_01""","""I""",3,493,36,0.8,-974,499,1.57,-0.423,245,216,0.12,2012,-3975,"""adult""",63,63.1,"""cross"""


Now, we can highlight the strike with the highest impact force for each frog. To do this, we use a windowing function to return the `True` is the impact force is equal to the maximal impact force for a given frog. This is used to specify the rows we highlight.

In [7]:
rows = (pl.col("impact force (mN)") == pl.col("impact force (mN)").max()).over("ID")

(
    df.style.tab_options(table_font_size="x-small")
    .tab_header(title="Frog tongue strikes")
    .tab_style(
        style=great_tables.style.fill("#99dbc9"),
        locations=great_tables.loc.body(rows=rows),
    )
)

GT(_tbl_data=shape: (80, 19)
┌────────────┬─────┬────────┬───────────────────┬───┬──────────┬──────────┬────────────┬───────────┐
│ date       ┆ ID  ┆ trial  ┆ impact force (mN) ┆ … ┆ age      ┆ SVL (mm) ┆ weight (g) ┆ species   │
│ ---        ┆ --- ┆ number ┆ ---               ┆   ┆ ---      ┆ ---      ┆ ---        ┆ ---       │
│ str        ┆ str ┆ ---    ┆ i64               ┆   ┆ str      ┆ i64      ┆ f64        ┆ str       │
│            ┆     ┆ i64    ┆                   ┆   ┆          ┆          ┆            ┆           │
╞════════════╪═════╪════════╪═══════════════════╪═══╪══════════╪══════════╪════════════╪═══════════╡
│ 2013_02_26 ┆ I   ┆ 3      ┆ 1205              ┆ … ┆ adult    ┆ 63       ┆ 63.1       ┆ cross     │
│ 2013_02_26 ┆ I   ┆ 4      ┆ 2527              ┆ … ┆ adult    ┆ 63       ┆ 63.1       ┆ cross     │
│ 2013_03_01 ┆ I   ┆ 1      ┆ 1745              ┆ … ┆ adult    ┆ 63       ┆ 63.1       ┆ cross     │
│ 2013_03_01 ┆ I   ┆ 2      ┆ 1556              ┆ … ┆ adult    ┆ 63       ┆ 63.1       ┆ cross     │
│ 2013_03_01 ┆ I   ┆ 3      ┆ 493               ┆ … ┆ adult    ┆ 63       ┆ 63.1       ┆ cross     │
│ …          ┆ …   ┆ …      ┆ …                 ┆ … ┆ …        ┆ …        ┆ …          ┆ …         │
│ 2013_06_18 ┆ IV  ┆ 4      ┆ 402               ┆ … ┆ juvenile ┆ 31       ┆ 12.7       ┆ cranwelli │
│ 2013_06_21 ┆ IV  ┆ 1      ┆ 605               ┆ … ┆ juvenile ┆ 31       ┆ 12.7       ┆ cranwelli │
│ 2013_06_21 ┆ IV  ┆ 2      ┆ 711               ┆ … ┆ juvenile ┆ 31       ┆ 12.7       ┆ cranwelli │
│ 2013_06_21 ┆ IV  ┆ 3      ┆ 614               ┆ … ┆ juvenile ┆ 31       ┆ 12.7       ┆ cranwelli │
│ 2013_06_21 ┆ IV  ┆ 4      ┆ 468               ┆ … ┆ juvenile ┆ 31       ┆ 12.7       ┆ cranwelli │
└────────────┴─────┴────────┴───────────────────┴───┴──────────┴──────────┴────────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x10423ac90>, _boxhead=Boxhead([ColInfo(var='date', type=<ColInfoTypeEnum.default: 1>, column_label='date', column_align='left', column_width=None), ColInfo(var='ID', type=<ColInfoTypeEnum.default: 1>, column_label='ID', column_align='left', column_width=None), ColInfo(var='trial number', type=<ColInfoTypeEnum.default: 1>, column_label='trial number', column_align='right', column_width=None), ColInfo(var='impact force (mN)', type=<ColInfoTypeEnum.default: 1>, column_label='impact force (mN)', column_align='right', column_width=None), ColInfo(var='impact time (ms)', type=<ColInfoTypeEnum.default: 1>, column_label='impact time (ms)', column_align='right', column_width=None), ColInfo(var='impact force / body weight', type=<ColInfoTypeEnum.default: 1>, column_label='impact force / body weight', column_align='right', column_width=None), ColInfo(var='adhesive force (mN)', type=<ColInfoTypeEnum.default: 1>, column_label='adhesive force (mN)', column_align='right', column_width=None), ColInfo(var='time frog pulls on target (ms)', type=<ColInfoTypeEnum.default: 1>, column_label='time frog pulls on target (ms)', column_align='right', column_width=None), ColInfo(var='adhesive force / body weight', type=<ColInfoTypeEnum.default: 1>, column_label='adhesive force / body weight', column_align='right', column_width=None), ColInfo(var='adhesive impulse (N-s)', type=<ColInfoTypeEnum.default: 1>, column_label='adhesive impulse (N-s)', column_align='right', column_width=None), ColInfo(var='total contact area (mm2)', type=<ColInfoTypeEnum.default: 1>, column_label='total contact area (mm2)', column_align='right', column_width=None), ColInfo(var='contact area without mucus (mm2)', type=<ColInfoTypeEnum.default: 1>, column_label='contact area without mucus (mm2)', column_align='right', column_width=None), ColInfo(var='contact area with mucus / contact area without mucus', type=<ColInfoTypeEnum.default: 1>, column_label='contact area with mucus / contact area without mucus', column_align='right', column_width=None), ColInfo(var='contact pressure (Pa)', type=<ColInfoTypeEnum.default: 1>, column_l

## Computing environment

In [8]:
%load_ext watermark
%watermark -v -p polars,jupyterlab

Python implementation: CPython
Python version       : 3.12.9
IPython version      : 9.1.0

polars    : 1.29.0
jupyterlab: 4.3.7

